In [1]:
import pandas as pd
import numpy as np
import requests
import os
from urllib.request import urlretrieve
from google.cloud import storage
import time


In [2]:
curr_dir = os.getcwd()
parent_dir = os.path.dirname(curr_dir)

names_csv = os.path.join(parent_dir, 'raw_data', 'Common_Edible_Mushrooms.csv')

In [3]:
columns = ['Common name', 'Scientific name']
df = pd.read_csv(names_csv, header=None, names = columns, skiprows=1, encoding='windows-1252')

In [4]:
df

,Common name,Scientific name
0,Almond mushroom,Agaricus subrufescens
1,Amethyst Chanterelle,Cantharellus amethysteus
2,Amethyst Deceiver,Laccaria amethystina
3,Aniseed Funnel,Clitocybe odora
4,Ascot Hat,Hortiboletus bubalinus
...,...,...
165,Wood Hedgehog,Hydnum repandum
166,Yellow Foot Waxcap,Cuphophyllus flavipes
167,Yellow Knight Mushroom,Tricholoma equestre
168,Yellow Swamp Brittlegill,Russula claroflava


In [5]:
species_list = df['Scientific name'].tolist()
species_list = sorted(species_list)
species_list

['Agaricus Iangei',
 'Agaricus arvensis',
 'Agaricus augustus',
 'Agaricus bisporus',
 'Agaricus bisporus (mature form)',
 'Agaricus bitorquis',
 'Agaricus bohusii',
 'Agaricus campestris',
 'Agaricus crocodilinus',
 'Agaricus langei',
 'Agaricus subrufescens',
 'Agaricus sylvaticus',
 'Agaricus sylvicola',
 'Agrocybe praecox',
 'Aleuria aurantia',
 'Amanita caesarea',
 'Amanita ceciliae',
 'Amanita crocea',
 'Amanita excelsa',
 'Amanita fulva',
 'Amanita rubescens',
 'Amanita rubescens',
 'Amanita sect. Vaginatae',
 'Apioperdon pyriforme',
 'Armillaria mellea',
 'Aspropaxillus giganteus',
 'Auricularia auricula-judae',
 'Auricularia auricula-judae',
 'Auricularia cornea',
 'Boletus aereus',
 'Boletus edulis',
 'Boletus edulis',
 'Boletus pinophilus',
 'Boletus reticulatus',
 'Butyriboletus appendiculatus',
 'Calocybe gambosa',
 'Calvatia gigantea',
 'Cantharellus amethysteus',
 'Cantharellus cibarius',
 'Cantharellus cinnabarinus',
 'Cantharellus friesii',
 'Cantharellus pallens',
 'C

In [6]:
print(len(species_list))
temp =pd.DataFrame({'species':species_list})
temp

170


,species
0,Agaricus Iangei
1,Agaricus arvensis
2,Agaricus augustus
3,Agaricus bisporus
4,Agaricus bisporus (mature form)
...,...
165,Volvopluteus gloiocephalus
166,Xerocomellus chrysenteron
167,Xerocomellus porosporus
168,Xerocomellus pruinatus


In [7]:
temp = temp.drop_duplicates()
temp


,species
0,Agaricus Iangei
1,Agaricus arvensis
2,Agaricus augustus
3,Agaricus bisporus
4,Agaricus bisporus (mature form)
...,...
165,Volvopluteus gloiocephalus
166,Xerocomellus chrysenteron
167,Xerocomellus porosporus
168,Xerocomellus pruinatus


In [8]:
species_list_new = temp.species.tolist()
species_list_new

['Agaricus Iangei',
 'Agaricus arvensis',
 'Agaricus augustus',
 'Agaricus bisporus',
 'Agaricus bisporus (mature form)',
 'Agaricus bitorquis',
 'Agaricus bohusii',
 'Agaricus campestris',
 'Agaricus crocodilinus',
 'Agaricus langei',
 'Agaricus subrufescens',
 'Agaricus sylvaticus',
 'Agaricus sylvicola',
 'Agrocybe praecox',
 'Aleuria aurantia',
 'Amanita caesarea',
 'Amanita ceciliae',
 'Amanita crocea',
 'Amanita excelsa',
 'Amanita fulva',
 'Amanita rubescens',
 'Amanita sect. Vaginatae',
 'Apioperdon pyriforme',
 'Armillaria mellea',
 'Aspropaxillus giganteus',
 'Auricularia auricula-judae',
 'Auricularia cornea',
 'Boletus aereus',
 'Boletus edulis',
 'Boletus pinophilus',
 'Boletus reticulatus',
 'Butyriboletus appendiculatus',
 'Calocybe gambosa',
 'Calvatia gigantea',
 'Cantharellus amethysteus',
 'Cantharellus cibarius',
 'Cantharellus cinnabarinus',
 'Cantharellus friesii',
 'Cantharellus pallens',
 'Cerioporus squamosus',
 'Chalciporus piperatus',
 'Chlorophyllum rhacodes

In [8]:
# save images to files in google cloud from a local file
# could not get this to work, find out why...(?)

def upload_to_gcs(bucket_name, source_file, blob_name):
    try:
        client = storage.Client(project=os.environ.get('GCP_PROJECT_ID'),credentials=os.environ.get["GOOGLE_APPLICATION_CREDENTIAL"])
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(source_file)
        print(f'File {source_file} uploaded to GCS bucket {bucket_name} with blob {blob_name}')
    except Exception as e:
        print(f'failed to upload {source_file} to {blob_name}: {e}')


In [9]:
# does not work - find out why
# build a function to use with the iNaturalist_API

def get_inaturalist_images_and_upload(taxon_name, bucket_name, num_results=500, image_size = 'original'):

    url = 'https://api.inaturalist.org/v1/observations'

#     query parameters

    params = {

        'q' : taxon_name,
        'per_page': num_results, #(1-200)
        'has_photos': 'true',
        'photo_license': 'any',
        'quality_grade': 'research',
        'order': 'desc',
        'order_by': 'created_at'
        }


    response = requests.get(url, params = params)

#     check if successful

    if response.status_code == 200:
        data = response.json()
        observations = data['results']

        # for i,obs in enumerate(observations):
        #     if 'photos' in obs:
        #         print(f"Observation {i+1} has {len(obs['photos'])} photos:")
        #         for j, photo in enumerate(obs['photos']):
        #             print(f" Photo {j+1}: {photo['url']}")
        #     else:
        #         print(f'Pbservation {i+1} has no photos.')


        # create a local temp folder to save results temporarily
        temp_dir = f"temp_images_{taxon_name.replace(' ','_')}"
        if not os.path.exists(temp_dir):
            os.makedirs(temp_dir)

#         collect and download

        image_urls = []
        for obs in observations:
            if 'photos' in obs:
                for i,photo in enumerate(obs['photos']):
                    try:
                        if 'url' not in photo or not isinstance(photo['url'],str):
                            print(f'Photo {i+1} for {taxon_name} does not have a valid URL..Skipping..')
                            continue

                        img_url = photo['url']
                        if isinstance(img_url,str) and 'square' in img_url:
                            img_url = img_url.replace('square', image_size)

                        print(f'Attempting to download from: {img_url}')

                        image_urls.append(img_url)

                        # dowload locally
                        local_image_path = os.path.join(temp_dir, f"{taxon_name.replace(' ','_')}_img_{i+1}.jpg")
                        urlretrieve(img_url, local_image_path)

                        # create subfolder for GCS
                        gcs_destination = f"{taxon_name.replace(' ','_')}/{os.path.basename(local_image_path)}"

                        # upload to GCS
                        upload_to_gcs(bucket_name, local_image_path, gcs_destination)

                    except Exception as e:
                        print(f'Error processing photo {i+1} for {taxon_name}: {e}')
                        continue

                    # Cleanup: Remove the temporary local directory after upload

        for file in os.listdir(temp_dir):
            os.remove(os.path.join(temp_dir, file))
        os.rmdir(temp_dir)

            # Return the number of images downloaded for this species
        return len(image_urls)

    else:
        raise Exception(f"Error fetching data for {taxon_name}: {response.status_code}")

In [11]:
# does not work - find out why
# Function to process a list of species names and log image count

def process_species_list(species_list, bucket_name, num_results=500, image_size='medium', progress_file="processed_species.txt"):

    """Processes each species in the list, retrieves and uploads images, and tracks progress."""
    # Load the list of already processed species
    processed_species = load_processed_species(progress_file)
    image_count_log = {}  # Dictionary to log the number of images retrieved per species

    # Process each species
    for species in species_list:
        if species not in processed_species:
            print(f"Processing species: {species}")
            try:
                # Get images and upload them, also return the count of images downloaded
                num_images = get_inaturalist_images_and_upload(species, bucket_name, num_results, image_size)

                # Log the number of images retrieved for this species
                image_count_log[species] = num_images

                # Save progress
                save_processed_species(progress_file, species)

                # Wait for 5 seconds before processing the next species
                time.sleep(5)

            except Exception as e:
                print(f"Error processing {species}: {e}")
                print("Stopping process to allow for investigation.")
                break  # Stop the process in case of an error
        else:
            print(f"Skipping already processed species: {species}")

    # Output how many images were downloaded for each species
    print("\nImage download summary:")
    for species, count in image_count_log.items():
        print(f"{species}: {count} images")

In [18]:
# does not work - find out why

test_list = ['Agaricus subrufescens','Cantharellus amethysteus']
bucket_name='edible_mushroom_images'


process_species_list(test_list, bucket_name, num_results=1, image_size = 'medium')


Processing species: Agaricus subrufescens
Attempting to download from: https://inaturalist-open-data.s3.amazonaws.com/photos/388745776/medium.jpg
failed to upload temp_images_Agaricus_subrufescens/Agaricus_subrufescens_img_1.jpg to Agaricus_subrufescens/Agaricus_subrufescens_img_1.jpg: 'method' object is not subscriptable
Attempting to download from: https://inaturalist-open-data.s3.amazonaws.com/photos/388746142/medium.jpg
failed to upload temp_images_Agaricus_subrufescens/Agaricus_subrufescens_img_2.jpg to Agaricus_subrufescens/Agaricus_subrufescens_img_2.jpg: 'method' object is not subscriptable
Processing species: Cantharellus amethysteus
Attempting to download from: https://static.inaturalist.org/photos/438598702/medium.jpeg
failed to upload temp_images_Cantharellus_amethysteus/Cantharellus_amethysteus_img_1.jpg to Cantharellus_amethysteus/Cantharellus_amethysteus_img_1.jpg: 'method' object is not subscriptable
Attempting to download from: https://static.inaturalist.org/photos/438

In [13]:
# this works ! use this!

import os
import requests
from urllib.request import urlretrieve
import subprocess

def download_and_upload_to_gcs(taxon_name, total_images=10, bucket_name='edible_mushroom_images'):
    url = 'https://api.inaturalist.org/v1/observations'
    images_downloaded = 0
    page = 1

    # Create a temporary folder for the species
    local_folder = os.path.join(os.getcwd(), taxon_name)
    os.makedirs(local_folder, exist_ok=True)

    while images_downloaded < total_images:
        # Define query parameters with paging
        params = {
            'q': taxon_name,
            'has_photos': 'true',
            'photo_license': 'any',
            'quality_grade': 'research',
            'order': 'desc',
            'order_by': 'created_at',
            'page': page,
            'per_page': 100  # Fetch maximum observations per page (100)
        }

        # Make the API request
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code != 200:
            raise Exception(f"Error fetching data for {taxon_name}: {response.status_code}")

        # Parse the response JSON
        observations = response.json()['results']

        # Stop if no more observations are found
        if not observations:
            print("No more observations found.")
            break

        count = 1
        # Loop through observations and download medium-sized images
        for obs in observations:
            if 'photos' in obs:
                for photo in obs['photos']:
                    if images_downloaded >= total_images:
                        break  # Stop once we've downloaded the desired number of images

                    img_url = photo['url'].replace('square', 'medium')  # Get medium-sized image
                    local_filename = os.path.join(local_folder, f"{taxon_name}_img_{count}.jpg")
                    urlretrieve(img_url, local_filename)
                    images_downloaded += 1
                    print(f"Downloaded {local_filename}")
                    count += 1
        # Move to the next page
        page += 1

    # After downloading, use gsutil to upload to Google Cloud Storage
    if images_downloaded > 0:
        try:
            # Use gsutil to upload the folder to GCS bucket
            subprocess.run(['gsutil', '-m', 'cp', '-r', local_folder, f'gs://{bucket_name}/'], check=True)
            print(f"Successfully uploaded {local_folder} to gs://{bucket_name}/")
        except subprocess.CalledProcessError as e:
            print(f"Failed to upload to Google Cloud: {e}")
        finally:
            # Clean up the local folder after upload
            for file in os.listdir(local_folder):
                os.remove(os.path.join(local_folder, file))
            os.rmdir(local_folder)
            print(f"Cleaned up local folder: {local_folder}")


In [16]:
# Test case

download_and_upload_to_gcs('Xerocomellus porosporus', total_images=300, bucket_name='edible_mushroom_images')


Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus/Xerocomellus porosporus_img_1.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus/Xerocomellus porosporus_img_2.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus/Xerocomellus porosporus_img_3.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus/Xerocomellus porosporus_img_4.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus/Xerocomellus porosporus_img_5.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus/Xerocomellus porosporus_img_6.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus/Xerocomellus porosporus_img_7.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus/Xerocomellus porosporus_img_8.jpg
Downloaded /User

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying file:///Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus/Xerocomellus porosporus_img_239.jpg [Content-Type=image/jpeg]...
Copying file:///Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus/Xerocomellus porosporus_img_16.jpg [Content-Type=image/jpeg]...
Copying file:///Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus/Xerocomellus porosporus_img_205.jpg [Content-Type=image/jpeg]...
Copying file:///Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus/Xerocomellus porosporus_img_211.jpg [Content-Type=image/jpeg]...
Copying file:///Us

Successfully uploaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus to gs://edible_mushroom_images/
Cleaned up local folder: /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/Xerocomellus porosporus


In [14]:
# this worked, commented out here to stop it being run, upon loading this notebook and running all cells etc..
# Loop through the list of species and call the function for each taxon_name

# for taxon_name in species_list_new:
#     try:
#         download_and_upload_to_gcs(taxon_name, total_images=300, bucket_name='edible_mushroom_images')
#         time.sleep(10)  # Optional: add delay to avoid overwhelming the API
#     except Exception as e:
#         print(f"Error processing {taxon_name}: {e}")

Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/X/X_img_1.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/X/X_img_2.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/X/X_img_3.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/X/X_img_4.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/X/X_img_5.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/X/X_img_6.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/X/X_img_7.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/X/X_img_8.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/X/X_img_9.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/X/X_img_10.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/X/X_img_11.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mushrooms/notebooks/X/X_img_12.jpg
Downloaded /Users/rakeshpatel/code/TheRaksta/Mush

KeyboardInterrupt: 

In [32]:
# upload directly to GCS
# this does not work

import requests
from urllib.request import urlopen
from google.cloud import storage
import io

def upload_to_gcs(bucket_name, blob_name, image_data):

    """Uploads image data directly to Google Cloud Storage"""
    client = storage.Client(project=os.environ.get('GCP_PROJECT_ID'),credentials=os.environ.get["GOOGLE_APPLICATION_CREDENTIAL"])  # Use default credentials
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_file(image_data, content_type='image/jpeg')
    print(f"Uploaded {blob_name} to gs://{bucket_name}/")

def download_and_upload_to_gcs_2(taxon_name, total_images=300, bucket_name='edible_mushroom_images'):
    url = 'https://api.inaturalist.org/v1/observations'
    images_downloaded = 0
    page = 1

    while images_downloaded < total_images:
        # Define query parameters with paging
        params = {
            'q': taxon_name,
            'has_photos': 'true',
            'photo_license': 'any',
            'quality_grade': 'research',
            'order': 'desc',
            'order_by': 'created_at',
            'page': page,
            'per_page': 100  # Fetch maximum observations per page (100)
        }

        # Make the API request
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code != 200:
            raise Exception(f"Error fetching data for {taxon_name}: {response.status_code}")

        # Parse the response JSON
        observations = response.json()['results']

        # Stop if no more observations are found
        if not observations:
            print("No more observations found.")
            break

        # Loop through observations and upload images directly to GCS
        for obs in observations:
            if 'photos' in obs:
                for photo in obs['photos']:
                    if images_downloaded >= total_images:
                        break  # Stop once we've downloaded the desired number of images

                    # Get medium-sized image URL
                    img_url = photo['url'].replace('square', 'medium')
                    img_name = f"{taxon_name}_img_{images_downloaded + 1}.jpg"

                    # Open the image directly from the URL
                    with urlopen(img_url) as image_response:
                        image_data = io.BytesIO(image_response.read())
                        upload_to_gcs(bucket_name, img_name, image_data)  # Upload directly to GCS

                    images_downloaded += 1
                    print(f"Downloaded and uploaded {img_name}")

        # Move to the next page
        page += 1
